In [3]:
from util import file2labels
from collections import Counter

# Dev: Positive data size = 199; Negetive data size = 1895

preds = file2labels("predictions/baseline_0.48253968253968244.txt")

Counter(preds)

Counter({1: 431, 0: 1663})

In [4]:
preds = file2labels("predictions/bart_baseline_0.4925619834710743.txt")

Counter(preds)

Counter({1: 406, 0: 1688})

In [5]:
preds = file2labels("predictions/bart_augment_0.5306122448979591.txt")

Counter(preds)

Counter({1: 193, 0: 1901})

In [6]:
preds = file2labels("predictions/bart_augment_0.31768953068592054.txt")

Counter(preds)

Counter({1: 78, 0: 2016})

In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
TXT = "My friends are <mask> but they eat too many carbs."

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
input_ids = tokenizer([TXT, TXT], return_tensors="pt")
print(input_ids)

with tokenizer.as_target_tokenizer():
    input_ids = tokenizer([TXT, TXT], return_tensors="pt")
    print(input_ids)

{'input_ids': tensor([[    0,  2387,   964,    32, 50264,    53,    51,  3529,   350,   171,
         33237,     4,     2],
        [    0,  2387,   964,    32, 50264,    53,    51,  3529,   350,   171,
         33237,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[    0,  2387,   964,    32, 50264,    53,    51,  3529,   350,   171,
         33237,     4,     2],
        [    0,  2387,   964,    32, 50264,    53,    51,  3529,   350,   171,
         33237,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [23]:
logits = model(input_ids).logits

masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)

tokenizer.decode(predictions).split()

['not', 'good', 'healthy', 'great', 'very']

In [18]:
tokenizer([TXT], return_tensors="pt")

{'input_ids': tensor([[    0,  2387,   964,    32, 50264,    53,    51,  3529,   350,   171,
         33237,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
yes_tok = tokenizer.convert_tokens_to_ids(["patronizing", "respectful"])
yes_tok

[3, 3]

In [8]:
tokenizer.decode(3).split()

['<unk>']

In [21]:
probs[9904], probs[3084]

(tensor(2.6303e-08, grad_fn=<SelectBackward0>),
 tensor(2.1232e-07, grad_fn=<SelectBackward0>))

In [17]:
from dont_patronize_me import DontPatronizeMe
import pandas as pd
from data_loader import rebuild_raw_dataset

dpm = DontPatronizeMe("data", "")

dpm.load_task1()
dpm.load_task2()

task1df = dpm.train_task1_df
task2df = dpm.train_task2_df

task2df

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


,par_id,art_id,text,keyword,country,label
0,4046,@@14767805,We also know that they can benefit by receivin...,hopeless,us,"[1, 0, 0, 1, 0, 0, 0]"
1,1279,@@7896098,Pope Francis washed and kissed the feet of Mus...,refugee,ng,"[0, 1, 0, 0, 0, 0, 0]"
2,8330,@@17252299,Many refugees do n't want to be resettled anyw...,refugee,ng,"[0, 0, 1, 0, 0, 0, 0]"
3,4063,@@3002894,"""Budding chefs , like """" Fred """" , """" Winston ...",in-need,ie,"[1, 0, 0, 1, 1, 1, 0]"
4,4089,@@25597822,"""In a 90-degree view of his constituency , one...",homeless,pk,"[1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...
988,873,@@20374243,Citing the fact that these kids who died at Go...,poor-families,sg,"[1, 0, 0, 0, 1, 0, 0]"
989,10070,@@15573661,Fern ? ndez was a well-known philanthropist wh...,disabled,ng,"[1, 0, 0, 0, 0, 0, 0]"
990,6484,@@2559173,"Touched much by their plight , Commanding Offi...",homeless,lk,"[1, 0, 0, 0, 0, 1, 0]"
991,6249,@@1947926,She reiterated her ministry 's commitment to p...,women,gh,"[1, 0, 0, 0, 0, 0, 0]"


In [18]:
intersect_df = pd.merge(task1df, task2df, how ='inner', on =["text"])
# intersect_df = intersect_df.drop(columns=['par_id_y', 'art_id_y', 'keyword_y', 'country_y', 'label_y'])
# intersect_df = intersect_df.rename(columns={"par_id_x": "par_id", "art_id_x": "art_id", "keyword_x": "keyword", "country_x": "country", "label_x": "label"})
intersect_df

,par_id_x,art_id_x,keyword_x,country_x,text,label_x,orig_label,par_id_y,art_id_y,keyword_y,country_y,label_y
0,33,@@8301323,disabled,pk,Arshad said that besides learning many new asp...,1,2,33,@@8301323,disabled,pk,"[1, 0, 0, 0, 0, 0, 0]"
1,34,@@24150149,disabled,ng,Fast food employee who fed disabled man become...,1,3,34,@@24150149,disabled,ng,"[1, 1, 0, 0, 0, 0, 0]"
2,42,@@4591694,hopeless,jm,Vanessa had feelings of hopelessness in her fi...,1,3,42,@@4591694,hopeless,jm,"[0, 0, 0, 0, 0, 1, 0]"
3,77,@@22454828,homeless,nz,"In September , Major Nottle set off on foot fr...",1,3,77,@@22454828,homeless,nz,"[1, 1, 0, 0, 0, 0, 0]"
4,83,@@4672144,homeless,pk,The demographics of Pakistan and India are ver...,1,3,83,@@4672144,homeless,pk,"[1, 1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
988,10445,@@3923193,refugee,gb,More than 150 volunteers spent the night in ' ...,1,3,10445,@@3923193,refugee,gb,"[0, 1, 0, 0, 0, 0, 0]"
989,10454,@@22338535,vulnerable,ie,""""""" We are challenged , I suggest , to turn th...",1,4,10454,@@22338535,vulnerable,ie,"[1, 1, 0, 0, 1, 1, 0]"
990,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3,10467,@@20282330,in-need,ng,"[1, 0, 0, 0, 0, 0, 0]"
991,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4,10468,@@16753236,hopeless,in,"[1, 0, 1, 0, 0, 1, 0]"


In [20]:
from dont_patronize_me import DontPatronizeMe
import pandas as pd


dpm = DontPatronizeMe('data', '.')
dpm.load_task1("test.tsv")

data = dpm.train_task1_df
data

,par_id,art_id,keyword,country,text,label,orig_label
0,4046,@@14767805,hopeless,us,We also know that they can benefit by receivin...,1,3
1,1279,@@7896098,refugee,ng,Pope Francis washed and kissed the feet of Mus...,1,4
2,8330,@@17252299,refugee,ng,Many refugees do n't want to be resettled anyw...,1,2
3,4063,@@3002894,in-need,ie,"""Budding chefs , like """" Fred """" , """" Winston ...",1,4
4,4089,@@25597822,homeless,pk,"""In a 90-degree view of his constituency , one...",1,3
...,...,...,...,...,...,...,...
2089,10462,@@22092971,homeless,gh,"The sad spectacle , which occurred on Saturday...",0,0
2090,10463,@@4676355,refugee,pk,""""""" The Pakistani police came to our house and...",0,0
2091,10464,@@19612634,disabled,ie,"""When Marie O'Donoghue went looking for a spec...",0,0
2092,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1


In [21]:
keywords = set(data.keyword.to_list())
countries = set(data.country.to_list())
keywords, countries

({'disabled',
  'homeless',
  'hopeless',
  'immigrant',
  'in-need',
  'migrant',
  'poor-families',
  'refugee',
  'vulnerable',
  'women'},
 {'au',
  'bd',
  'ca',
  'gb',
  'gh',
  'hk',
  'ie',
  'in',
  'jm',
  'ke',
  'lk',
  'my',
  'ng',
  'nz',
  'ph',
  'pk',
  'sg',
  'tz',
  'us',
  'za'})

In [22]:
data = data[data.label==1]
# posdf

for k in keywords:
  print(f"{k}: {len(data[data.keyword==k])}")

for k in countries:
  print(f"{k}: {len(data[data.country==k])}")

homeless: 29
hopeless: 26
migrant: 5
disabled: 14
women: 14
vulnerable: 20
refugee: 13
in-need: 33
immigrant: 7
poor-families: 38
lk: 9
sg: 4
ph: 15
tz: 11
us: 10
ng: 15
in: 7
au: 5
gh: 12
ie: 10
ke: 13
bd: 7
pk: 14
gb: 9
my: 8
nz: 12
hk: 9
za: 11
ca: 8
jm: 10
